## Missing Values and Imputing

### Причины NA:  
   1. **Data is Lost:** частая проблема в производственной дате. Компания меняет сервера, обрастает бюрократией, цифровизуется медленно и поэтапно -> данные будут теряться.
   2. **Data is not captured:** нормальная история. Например, респондент не захотел отвечать на вопрос, или метрика была придуманна посреди процесса сбора данных, а перезапускать алгоритм нерентабельно.
   3. **Wrong values present in Dataset:** когда значения являются явными аутлейерами даже в предполагаемой генеральной совокупности. Например, возраст 221 год, город Москав, и т.д. (зачастую неверно работающие алгоритмы присылают случайные данные или нули, хотя на самом деле это именно NA)
   
Ещё одно разделение:  
   1. **Missing completely at random**
   2. **Missing at Random**
   3. **Missing not at random**
   
*The more data is missing (MNAR), the more biased are the estimations*. Именно поэтому в задачах МЛ важно обращать внимание на пропущенные данные в датасете перед *тренировкой* моделей. Более того: многие пакеты питона плохо работают с пропусками (попробуйте применить метод `.mean` к `numpy`евскому массиву c na, и вы удивитесь).

### Методы работы с NA (на примере [датасета с пингвинами](https://www.kaggle.com/parulpandey/penguin-dataset-the-new-iris))

In [1]:
! pip install palmerpenguins

In [2]:
# гланвый чанк в селе
from palmerpenguins import load_penguins
import pandas as pd
import numpy as np 
import math

df = load_penguins()
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [3]:
print("Длина первоначального датасета:", len(df))

Длина первоначального датасета: 344


#### 0. **Оставить всё как есть** (если алгоритм ***может и умеет*** работать с пропусками)

#### 1. **Простой**: дропнуть пропущенные значения.  

Отбрасываем строки/колонки, содержащие хоть_одну/несколько/все NaN.

   **+ :** минимум когнитивной нагрузки, максимум результата.  
   **-- :** суммарно строк с пропущенными значениями может быть настолько много, что они съедают ваш датасет.

    NB: не забывайте об опции вызова справки по функции - возможно, там есть параметры, которые нужны именно вам!

In [4]:
# help(pd.DataFrame.dropna) # - cправка

df1 = df.dropna() # так мы дропаем все строки, содержащие хотя бы 1 NA
df1.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007


In [5]:
len(df1)

333

In [6]:
df.dropna(axis = 'columns', how = "any").head(5) # а так мы отбрасываем все *колонки*, где есть *хоть один* NA, иногда полезно

,species,island,year
0,Adelie,Torgersen,2007
1,Adelie,Torgersen,2007
2,Adelie,Torgersen,2007
3,Adelie,Torgersen,2007
4,Adelie,Torgersen,2007


#### 2. **Импутация**: маскируем чем-то похожим (средними)

Быстро заменяем NA медианными/средними/модными значениями с помощью `SimpleImputer`

   **+ :** быстро делает numeric data ухоженными.  
   **-- :** только numeric, при большом количестве подстановок растёт bias датасета.

    NB: для категориальных тоже можно использовать моду - просто подставляем самое частотное значение в пропуски

In [7]:
from sklearn.impute import SimpleImputer
# help(SimpleImputer) # - справка

x = np.array(df['body_mass_g'])
x = x.reshape(-1,1)

print("Средняя масса пингвига:", x.mean())

Средняя масса пингвига: nan


In [8]:
#стратегии: mean, median, most_frequent, constant

imputer = SimpleImputer(strategy = 'mean')
imputer.fit(x)
x1 = imputer.transform(x)

print("Средняя масса пингвига:", x1.mean())
# df['body_mass_g'] = imputer.transform(x)

Средняя масса пингвига: 4201.754385964912


#### 3. Импутация: метод k-NN (классика классификации)
**+** : прикольнее и разнообразнее, нежели простым импутером.  
**--** : компутация на больших датасетах займёт время.

In [9]:
from sklearn.impute import KNNImputer
# help(KNNImputer) # - справка

k = np.array(df['body_mass_g'])
k = x.reshape(-1,1)

print("Средняя масса пингвига:", k.mean())

Средняя масса пингвига: nan


In [10]:
# задаём - n_neighbors

imputer = KNNImputer(n_neighbors = 3, weights='uniform', metric='nan_euclidean')
k1 = imputer.fit_transform(k)

print("Средняя масса пингвига:", k1.mean())
# df['body_mass_g'] = imputer.transform(x)

Средняя масса пингвига: 4201.754385964912


Для датасета - сразу циклом:

In [11]:
data = df.select_dtypes(include=np.number).values
ix = [i for i in  range(data.shape[1])]
X = data[:, ix]
print('old Missing: %d' % sum(np.isnan(X).flatten()))

imputer = KNNImputer()
imputer.fit(X)
Xtrans = imputer.transform(X)

print('new Missing: %d' % sum(np.isnan(Xtrans).flatten()))

old Missing: 8
new Missing: 0


Кстати, ***категориальные*** переменные ***также можно импутить*** - достаточно вертануть их в дамми и обратно:

In [12]:
df['is_male'] = df['sex'].map({'male': 1,
                             'female': 0})
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,is_male
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007,1.0
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,0.0
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007,0.0
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007,0.0


In [13]:
sex = df['is_male'].values
sex = sex.reshape(-1,1)

imputer = KNNImputer(n_neighbors = 2)
sex1 = imputer.fit_transform(sex)
sex1

array([[1.       ],
       [0.       ],
       [0.       ],
       [0.5045045],
       [0.       ],
       [1.       ],
       [0.       ],
       [1.       ],
       [0.5045045],
       [0.5045045],
       [0.5045045],
       [0.5045045],
       [0.       ],
       [1.       ],
       [1.       ],
       [0.       ],
       [0.       ],
       [1.       ],
       [0.       ],
       [1.       ],
       [0.       ],
       [1.       ],
       [0.       ],
       [1.       ],
       [1.       ],
       [0.       ],
       [1.       ],
       [0.       ],
       [0.       ],
       [1.       ],
       [0.       ],
       [1.       ],
       [0.       ],
       [1.       ],
       [0.       ],
       [1.       ],
       [1.       ],
       [0.       ],
       [0.       ],
       [1.       ],
       [0.       ],
       [1.       ],
       [0.       ],
       [1.       ],
       [0.       ],
       [1.       ],
       [1.       ],
       [0.5045045],
       [0.       ],
       [1.       ],


*правда как интерпретировать гендер 0.5 я не знаю*

### Чё ещё почитать по теме:
 * [Медиум](https://towardsdatascience.com/the-robustness-of-machine-learning-algorithms-against-missing-or-abnormal-values-ec3222379905), где автор сравнивает перфоманс rf, boosting и lasso с разными методами инпутации;
 * [yet another Медиум](https://medium.com/analytics-vidhya/why-it-is-important-to-handle-missing-data-and-10-methods-to-do-it-29d32ec4e6a), где можно почитать описания методов;
 * [гайд](https://machinelearningmastery.com/knn-imputation-for-missing-values-in-machine-learning/) по kNN-у.